# Import Libraries

This was the first project I made in Kaggle. The goal here is to create a machine learning to predict which passengers survived the sinking of the RMS Titanic using the ship's passenger manifest. 

In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

2024-04-12 10:46:41.886792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 10:46:41.887002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 10:46:42.086177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load train data

In [2]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
print(train_df.info())
print(test_df.info())

train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data preparation

In [3]:
train_df['Pclass'] = train_df['Pclass'].astype('category')
test_df['Pclass'] = test_df['Pclass'].astype('category')


# fill missing numbers with median
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].median())

# get dummies for categorical variables
train_df_encoded = pd.get_dummies(train_df[['Sex','Embarked', 'Pclass']], drop_first = True)
train_df = pd.concat([train_df, train_df_encoded], axis=1)

test_df_encoded = pd.get_dummies(test_df[['Sex','Embarked', 'Pclass']], drop_first = True)
test_df = pd.concat([test_df, test_df_encoded], axis=1)

# change cabin variable to whether a the passanger had a cabin assigned (1) or had no cabin assigned (0)
def replace_non_null_and_nan(value):
    if pd.notna(value):
        return 1
    else:
        return 0

train_df['Cabin'] = train_df['Cabin'].apply(replace_non_null_and_nan)
test_df['Cabin'] = test_df['Cabin'].apply(replace_non_null_and_nan)

train_df['Cabin'] = train_df['Cabin'].astype('bool')
test_df['Cabin'] = test_df['Cabin'].astype('bool')

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,False,S,True,False,True,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,True,C,False,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,False,S,False,False,True,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,True,S,False,False,True,False,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,False,S,True,False,True,False,True


In [4]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,False,Q,True,True,False,False,True
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,False,S,False,False,True,False,True
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,False,Q,True,True,False,True,False
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,False,S,True,False,True,False,True
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,False,S,False,False,True,False,True


In [5]:
test_features = ['Sex_male', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S', 'Pclass_2', 'Pclass_3']

X = train_df[test_features]
y = train_df['Survived']
X_test = test_df[test_features]

print("X_train shape:", X.shape)
print("y_train shape:", y.shape)
print("X_test shape:", X_test.shape)

X_train shape: (891, 10)
y_train shape: (891,)
X_test shape: (418, 10)


In [6]:
# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
# Standardizing the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled =  scaler.transform(X_test)
# Initialize a Sequential model
model = Sequential()

# Add a dense layer with ReLU activation for the hidden layer
model.add(Dense(units=200, activation='relu', input_dim=X_train.shape[1]))

# Add the output layer with sigmoid activation for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_val_scaled, y_val))

# Predict on the validation set
y_val_pred = model.predict(X_val_scaled)
y_val_pred_binary = (y_val_pred > 0.5).astype(int)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_val, y_val_pred_binary)
print("Accuracy on Validation Set:", accuracy)

Epoch 1/10
23/23 [==============================] - 1s 15ms/step - loss: 0.6136 - accuracy: 0.7261 - val_loss: 0.5425 - val_accuracy: 0.7765
Epoch 2/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4886 - accuracy: 0.7921 - val_loss: 0.4807 - val_accuracy: 0.7821
Epoch 3/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4469 - accuracy: 0.7949 - val_loss: 0.4592 - val_accuracy: 0.8101
Epoch 4/10
23/23 [==============================] - 0s 5ms/step - loss: 0.4302 - accuracy: 0.8062 - val_loss: 0.4490 - val_accuracy: 0.8101
Epoch 5/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4200 - accuracy: 0.8132 - val_loss: 0.4423 - val_accuracy: 0.8156
Epoch 6/10
23/23 [==============================] - 0s 5ms/step - loss: 0.4116 - accuracy: 0.8104 - val_loss: 0.4381 - val_accuracy: 0.8156
Epoch 7/10
23/23 [==============================] - 0s 4ms/step - loss: 0.4061 - accuracy: 0.8287 - val_loss: 0.4346 - val_accuracy: 0.8212
Epoch 8/10
23/23 [=

# Predict on test data

In [8]:
# run model on test data
y_test_pred = model.predict(X_test_scaled)
# convert survival probabilities into binary outcome
y_test_pred_binary = (y_test_pred > 0.5).astype(int)

14/14 [==============================] - 0s 2ms/step


In [9]:
y_test_pred_binary
y_test_pred_binary = y_test_pred_binary.flatten()

In [10]:
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': y_test_pred_binary})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [11]:
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
